In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [6]:
# import pandas as pd
# # import os
# from datasets import load_dataset

# # cwd = os.getcwd()

# dataset = load_dataset('ag_news')
# print(dataset)

In [7]:
# df_train = pd.DataFrame(dataset['train'])
# # df_train : Index(['text', 'label'], dtype='object')
# df_test = pd.DataFrame(dataset['test'])

# # train data의 " "(띄워쓰기) 를 "_"로 변환
# # df_train.columns = df_train.columns.str.replace(" ","_")

# df_train['name'] = df_train['label'].map({0:"World", 1:"Sports", 2 : "Business", 3 :"Sci_Tech"})
# # print(df_train['text'][100],df_train['label'][100])

# for i in range(10):
#     print("Title of Article",i)
#     print(df_train.loc[i,"title"])
#     print("\n")

In [8]:
import pandas as pd
import os

data = pd.read_csv("/home/sky3alfory/NLP_prac/nlpbook/data/train.csv")
data = pd.DataFrame(data=data)
print(data)



        Class Index                                              Title  \
0                 3  Wall St. Bears Claw Back Into the Black (Reuters)   
1                 3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2                 3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3                 3  Iraq Halts Oil Exports from Main Southern Pipe...   
4                 3  Oil prices soar to all-time record, posing new...   
...             ...                                                ...   
119995            1  Pakistan's Musharraf Says Won't Quit as Army C...   
119996            2                  Renteria signing a top-shelf deal   
119997            2                    Saban not going to Dolphins yet   
119998            2                                  Today's NFL games   
119999            2                       Nets get Carter from Raptors   

                                              Description  
0       Reuters - Short-sellers, Wall Street's dwin

In [9]:
data.columns =data.columns.str.replace(" ","_")
data.columns = data.columns.str.lower()
data["class_name"] = data["class_index"].map({1:"World",2:"Sports",3:"Business",4:"Sci_Tech"})
data

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,World
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports


In [10]:
data.class_name.value_counts()

class_name
Business    30000
Sci_Tech    30000
Sports      30000
World       30000
Name: count, dtype: int64

In [11]:
for i in range(10):
    print("Title of Article",i)
    print(data.loc[i,"title"])
    print("Description of Article",i)
    print(data.loc[i,'description'])
    print("\n")

Title of Article 0
Wall St. Bears Claw Back Into the Black (Reuters)
Description of Article 0
Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


Title of Article 1
Carlyle Looks Toward Commercial Aerospace (Reuters)
Description of Article 1
Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


Title of Article 2
Oil and Economy Cloud Stocks' Outlook (Reuters)
Description of Article 2
Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.


Title of Article 3
Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)
Description of Article 3
Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence sh

In [12]:
cols = ["title","description"]
data[cols] = data[cols].applymap(lambda x: x.replace("\\"," "))
data[cols] = data[cols].applymap(lambda x: x.replace("#36","$"))
data[cols] = data[cols].applymap(lambda x: x.replace("  "," "))
data[cols] = data[cols].applymap(lambda x: x.strip())

data.to_csv("/home/sky3alfory/NLP_prac/nlpbook/data/train_prepared.csv", index=False)

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import spacy
# import torch

# device = torch.device("cuda:0")
spacy.require_gpu()
print(spacy.require_gpu())
nlp = spacy.load("en_core_web_trf")

ValueError: GPU is not accessible. Was the library installed correctly?

In [ ]:
from thinc.api import require_gpu
import cupy

require_gpu()  # GPU 활성화
print("현재 사용 중인 GPU ID:", cupy.cuda.runtime.getDevice())
print("총 사용 가능한 GPU:", cupy.cuda.runtime.getDeviceCount())

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(nlp.meta)

In [ ]:
nlp(data.loc[0,"description"])
# for i in range(9):
#     print("Article",i)
#     print(data.loc[i,"description"])
#     print("Text Start End Label")
#     doc = nlp(data.loc[i,"description"])
#     for token in doc.ents:
#         print(token.text, token.start_char,token.end_char, token.label)
#     print("\n")